In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split # used for splitting training and testing data
from sklearn.metrics import confusion_matrix

torch.manual_seed(1)

In [15]:
plt.figure(figsize=(8,5))

# time steps per batch of data
seq_length = 100

time_steps = np.linspace(0, np.pi, seq_length + 1)

data = pd.read_csv('../Datasets/SectionData.csv')

x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

<Figure size 576x360 with 0 Axes>

In [44]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, kernel_size, hidden_size, drop_prob):
        super(NeuralNetwork, self).__init__()
        self.model = nn.Sequential(
            nn.Conv1d(1, 1, kernel_size),
            nn.Flatten(),
            #nn.ReLU(),
            nn.Dropout(drop_prob),
            nn.Linear(input_size - kernel_size + 1, hidden_size[0]),
            nn.Linear(hidden_size[0], hidden_size[1]),
            nn.Linear(hidden_size[1], hidden_size[2]),
            nn.Softmax(dim=1),
        )
        #self.conv1 = nn.Conv1d(1, 1, kernel_size)
        #self.flat = nn.Flatten()
        #self.relu = nn.ReLU()
        #self.dropout = nn.Dropout(drop_prob)
        #self.fc1 = nn.Linear(input_size - kernel_size + 1, hidden_size[0])
        #self.fc2 = nn.Linear(hidden_size[0], hidden_size[1])
        #self.fc3 = nn.Linear(hidden_size[1], hidden_size[2])
        #self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        #print("X: " + str(x.shape))
        #out1 = self.conv1(x)
        #print("Out1: " + str(out1.size()))
        #outF = self.flat(out1)
        #print("OutF: " + str(out1.size()))
        #out2 = self.relu(outF)
        #print("Out2: " + str(out2.size()))
        #out3 = self.dropout(out2)
        #print("Out3: " + str(out3.size()))
        #out4 = self.fc1(out3)
        #print("Out4: " + str(out4.size()))
        #out5 = self.fc2(out4)
        #print("Out5: " + str(out5.size()))
        #out6 = self.fc3(out5)
        #print("Out6: " + str(out6.size()))
        #print(out6)
        #out7 = self.softmax(out6)
        #print("Out7: " + str(out7.size()))
        #print(out7)    
        
        out = self.model(x)
        
        return out

In [37]:
BEAT_TYPES_INDEX = {
    'N': 0,
    'L': 1,
    'R': 2,
    'A': 3,
    'V': 4,
    'F': 5,
}

def formatDataX(data):
    temp = torch.tensor(data).float()
    return temp.view(temp.size(0), 1, temp.size(1))

def formatDataY(data):
    return torch.tensor(data).float()

def preprocess(df):

    # drop ecgNum row
    df = df.drop(["ecgNum"], axis=1)
    # Classify the dependent and independent variables
    X = df.iloc[:, :-1].values
    Y = np.array([np.insert(np.zeros(5), BEAT_TYPES_INDEX[label], 1) for label in df.iloc[:, -1].values])
    
    # split the data into train, validate, test
    X_temp, X_test, Y_temp, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
    X_train, X_val, Y_train, Y_val = train_test_split(X_temp, Y_temp, test_size=0.2, random_state=0)
    
    return formatDataX(X_train), formatDataX(X_val), formatDataX(X_test), formatDataY(Y_train), formatDataY(Y_val), formatDataY(Y_test)


In [38]:
#X_train, X_val, X_test, Y_train, Y_val, Y_test = preprocess(data)
#print(X_train.size())
#print(X_val.size())
#print(X_test.size())
#print(Y_train.size())
#print(Y_val.size())
#print(Y_test.size())

In [45]:
# size of the input at each time step
# TODO: handle input sizes
input_size = 200
# kernel size
kernel_size = 20
# size of the hidden state and cell state at each time step
hidden_size = [64, 32, 6]
# dropout probability
drop_prob = 0.2

# instantiate the NN
neuralNet = NeuralNetwork(input_size, kernel_size, hidden_size, drop_prob)
print(neuralNet)

NeuralNetwork(
  (model): Sequential(
    (0): Conv1d(1, 1, kernel_size=(20,), stride=(1,))
    (1): Flatten()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=181, out_features=64, bias=True)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): Linear(in_features=32, out_features=6, bias=True)
    (6): Softmax(dim=1)
  )
)


In [46]:
# Mean Squared Error and Adam Optimizer with a learning rate of 0.01
# TODO: play with LR
#criterion = nn.MSELoss()
#optimizer = optim.SGD(net.parameters(), lr=0.01)
#optimizer = torch.optim.Adam(neuralNet.parameters(), lr=0.01)

In [47]:
# train the NN
def train(net, X, Y, num_epochs):  
    criterion = nn.MSELoss()
    #optimizer = optim.SGD(net.parameters(), lr=0.01)
    optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
    
    for epoch in range (num_epochs):
        optimizer.zero_grad()
        out = net(X)
        loss = criterion(out, Y)
        loss.backward()
        optimizer.step()
        print('Loss ', loss.item())

    return net

In [48]:
epochs = 100
print_every = 45

X_train, X_val, X_test, Y_train, Y_val, Y_test = preprocess(data)

In [49]:
trained_NN = train(neuralNet, X_train, Y_train, epochs)

Loss  0.1421380490064621
Loss  0.10222505033016205
Loss  0.07693897187709808
Loss  0.08315418660640717
Loss  0.08507984131574631
Loss  0.08560729026794434
Loss  0.08563666045665741
Loss  0.08525707572698593
Loss  0.08408773690462112
Loss  0.07980195432901382
Loss  0.08279348164796829
Loss  0.07527728378772736
Loss  0.07603474706411362
Loss  0.07439752668142319
Loss  0.07392140477895737
Loss  0.07331164926290512
Loss  0.07147959619760513
Loss  0.0745042935013771
Loss  0.07161404192447662
Loss  0.07313697785139084
Loss  0.07144074887037277
Loss  0.0718887448310852
Loss  0.07047086954116821
Loss  0.06954211741685867
Loss  0.0692533329129219
Loss  0.06856653839349747
Loss  0.06817538291215897
Loss  0.06833898276090622
Loss  0.06766212731599808
Loss  0.0673184022307396
Loss  0.06715712696313858
Loss  0.06677841395139694
Loss  0.06631115823984146
Loss  0.06615789979696274
Loss  0.06577794253826141
Loss  0.06532696634531021
Loss  0.0650155171751976
Loss  0.06467150151729584
Loss  0.0641918778

In [60]:
def test(net, X, Y):
    resValues, resIndices = torch.max(net(X), 1)
    length = len(resIndices.numpy())
    testValues, testIndices = torch.max(Y, 1)
    results = np.array([np.insert(np.zeros(5), index, 1) for index in resIndices])
    
    conf_matrix = confusion_matrix(resIndices.numpy(), testIndices.numpy())

    correct = 0
    for i in range(len(conf_matrix)):
        correct += conf_matrix[i,i]
        
    print("Accuracy: {:.2%}".format(correct/length))
    
    print("\nConfusion Matrix:")
    print(conf_matrix)

In [63]:
#test(trained_NN, X_train, Y_train)
test(trained_NN, X_test, Y_test)
#test(trained_NN, X_val, Y_val)

Accuracy: 83.24%

Confusion Matrix:
[[14731   927   493   267   629   145]
 [  125   495    11    22    84    10]
 [   85    32   938   144    62     0]
 [    4     1     0    10     2     0]
 [   90   174     4    46   558     0]
 [   10     2     0     0     0     0]]
